# Performance of LSH on coreferences

In [ ]:
from load_coreferences import load_coreferences
import lsh 
import copy
import numpy as np
import time 

import cProfile

scaling_factor = 5 # multiply size the original data set by this factor

In [ ]:
raw_mentions = load_coreferences()
mentions = {i: m for i, m in enumerate(raw_mentions)}


#### Effectiveness

#### Profiling

In [3]:

# stack mentions on top of each other 

mentions_scaled = copy.copy(mentions)

idx = len(mentions_scaled)
for i in range(1, scaling_factor):
    for idx_old in mentions.keys():
        m = mentions[idx_old]
        mentions_scaled[idx] = m 
        idx += 1

In [4]:
mylsh = lsh.LSHMinHash_nonp(mentions=mentions_scaled, shingle_size=3, signature_size=200, n_buckets=2)
cProfile.run("mylsh.cluster()")

         992045 function calls in 3.771 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.770    3.770 <string>:1(<module>)
        1    0.897    0.897    1.009    1.009 lsh.py:142(_min_hash)
        1    0.001    0.001    0.001    0.001 lsh.py:143(<dictcomp>)
        1    0.000    0.000    0.043    0.043 lsh.py:165(_make_bands)
        1    2.637    2.637    2.639    2.639 lsh.py:170(_make_clusters)
        1    0.000    0.000    0.000    0.000 lsh.py:171(<dictcomp>)
      870    0.040    0.000    0.043    0.000 lsh.py:19(partition_signature)
        1    0.000    0.000    3.770    3.770 lsh.py:190(cluster)
        1    0.000    0.000    0.000    0.000 lsh.py:67(_build_vocab)
        1    0.000    0.000    0.000    0.000 lsh.py:68(<listcomp>)
        1    0.000    0.000    0.000    0.000 lsh.py:69(<listcomp>)
        1    0.001    0.001    0.080    0.080 lsh.py:72(encode_binary)
      870    

In [5]:
mylsh = lsh.LSHMinHash(mentions=mentions_scaled, shingle_size=2, signature_size=200, band_length=2)

cProfile.run("mylsh.cluster()")

         214290 function calls (200381 primitive calls) in 0.397 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.000    0.000    0.003    0.000 <__array_function__ internals>:2(argsort)
      101    0.000    0.000    0.026    0.000 <__array_function__ internals>:2(array_split)
      103    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(concatenate)
      100    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(copyto)
      200    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(cumsum)
      100    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(diff)
      200    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(empty_like)
      200    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(fliplr)
      200    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(ndim)
      100    0.000    0.000   

In [ ]:
# trying out different configs 
    # --> it seems as soon as shingle size > 2 or band_length > 2, some candidates are missed. is this a feature or a bug?
mylsh = lsh.LSHMinHash(mentions=mentions_scaled, shingle_size=2, signature_size=50, band_length=2)

mylsh.cluster()
mylsh.summarise()
pairs = {mention: [mentions_scaled[i] for i in mylsh.candidates[idx]]  for idx, mention in mentions_scaled.items()}

pairs 

#### Time analysis for sorting multidimensional array along one axis

In [ ]:
# keep this for some time analysis
    # mult before sort -- adapt from the now helper function cols_to_int
x = np.array([[0, 3], [2, 2], [2, 2], [0, 3]]) 
x = np.array([[1, 3], [2, 2], [2, 2], [1, 3], [1, 5], [1, 1]]) # this is one test case
A = np.array([[3,4], [3,5], [5,6], [3,4], [6,7]]) # this is another test case

display(A)
display(np.argsort(A, axis=0))
display(A.view('i8,i8').argsort(order=['f1'], axis=0))

display(x)
display(np.argsort(x, axis=0))
display(x.view('i8,i8').argsort(order=['f1'], axis=0))


n = 100_000
A = np.random.choice(np.arange(int(n/100)), size=(n,2))
%timeit np.argsort(A, axis=0)
%timeit A.view('i8,i8').argsort(order=['f1'], axis=0)
%timeit mult_before_sort(A)